In [1]:
import sys
import os
sys.path.append('/home/dc755/Conjecturing/')
sys.path.append('../')
os.environ['PISA_PATH'] = '/home/dc755/Portal-to-ISAbelle/src/main/python'
import dsp_functions

checker = dsp_functions.Checker(
    working_dir='/home/dc755/Isabelle2022/src/HOL/Examples',
    isa_path='/home/dc755/Isabelle2022',
    theory_file='/home/dc755/Isabelle2022/src/HOL/Examples/Interactive.thy',
    port=8000
)

['/home/dc755/Conjecturing', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/dc755/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages', '/home/dc755/Conjecturing/', '../', '/home/dc755/Portal-to-ISAbelle/src/main/python']
intialised_env


In [3]:
import threading
import subprocess

def start_server():
    directory = "/home/dc755/Portal-to-ISAbelle/"
    command = "sbt \"runMain pisa.server.PisaOneStageServer8000\""
    subprocess.run(command, shell=True, cwd=directory, text=True)


server_thread = threading.Thread(target=start_server)

server_thread.start()

print("Server started in a background thread. Notebook can proceed to next cells.")


Server started in a background thread. Notebook can proceed to next cells.


[info] welcome to sbt 1.4.3 (Ubuntu Java 11.0.21)
[info] loading settings for project portal-to-isabelle-build from plugins.sbt ...
[info] loading project definition from /home/dc755/Portal-to-ISAbelle/project
[info] loading settings for project portal-to-isabelle from build.sbt ...
[info] set current project to PISA (in build file:/home/dc755/Portal-to-ISAbelle/)
[info] running pisa.server.PisaOneStageServer8000 
Server is running. Press Ctrl-C to stop.


In [ ]:
theorem_and_sledgehammer_proof = """
lemma "(1::nat) + 1 = 2"
    sledgehammer
"""
result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

In [ ]:
theorem_and_sledgehammer_proof = """theorem gcd_lcm:
  assumes "gcd (n :: nat) 4 = 1" 
      and "lcm (n :: nat) 4 = 28"
  shows "n = 7"
proof -
  have c1: "1*28 = n*4" using assms
    sledgehammer
  then have c2: "n = 1*28/4"
    sledgehammer
  then show ?thesis
    sledgehammer
qed"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

In [ ]:
theorem_and_sledgehammer_proof = """theorem
  fixes x::real
  assumes "0<x" "x<pi"
  shows "x * sin x > 0"
proof -
  show ?thesis
    sledgehammer
qed
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

In [ ]:
theorem_and_sledgehammer_proof = """
lemma y_positivity:
  fixes x::real
  assumes "0 < x" "x < pi"
  shows "x * sin x > 0"
proof -
  show ?thesis
    by (simp add: assms(1) assms(2) sin_gt_zero)
qed

lemma inequality_transformation:
  fixes y::real
  assumes "y > 0"
  shows "(9 * y^2 + 4) / y \<ge> 12"
proof -
  show ?thesis
  \<proof>
qed

theorem minimum_value_expression:
  fixes x::real
  assumes "0<x" "x<pi"
  shows "12 \<le> ((9 * (x^2 * (sin x)^2)) + 4) / (x * sin x)"
proof -
  define y where "y=x * sin x"
  have c1 : "y > 0"
    using assms y_positivity y_def
    sledgehammer
  then have c2: "(9 * y^2 + 4) / y \<ge> 12"
    using inequality_transformation y_def
    sledgehammer
  then show ?thesis
    using c1
    sledgehammer
qed
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

## Cascade

The declarative steps (in blue) and the lower-level proofs of those steps (in green) lead to a nice opportunity for a language model cascade.

Namely, Draft, Sketch, Prove uses a neural language model to "draft" an informal proof, "sketch" the declarative steps based on the informal proof, then attempts to "close the gaps" with Sledgehammer (i.e., prove each step). If the gaps are closed, the steps together with their proofs constitute a verified proof of the original theorem (and of course, this is checked by Isabelle).


## Draft and Sketch examples
To implement the drafting and sketching steps, we provide a few examples in the prompt.

We will derive these examples from ones used in the paper:

In [ ]:
import sys
sys.path.append('../')
import dsp_functions

import json
examples = [
    {"tag": "aimeI_2000_p7", "category": "algebra", "metadata": {}, "prompt": "Informal:\n(*### Problem\n\nSuppose that $x,$ $y,$ and $z$ are three positive numbers that satisfy the equations $xyz = 1,$ $x + \\frac {1}{z} = 5,$ and $y + \\frac {1}{x} = 29.$ Then $z + \\frac {1}{y} = \\frac {m}{n},$ where $m$ and $n$ are [[relatively prime]] positive integers. Find $m + n$. Show that it is 5.\n\n\nnote: this is the type of problem that makes you think symmetry, but actually can be solved easily with substitution, and other normal technniques\n\n### Solution\n\nWe can rewrite $xyz=1$ as $\\frac{1}{z}=xy$.\n\nSubstituting into one of the given equations, we have \n$x+xy=5$\n$x(1+y)=5$\n$\\frac{1}{x}=\\frac{1+y}{5}.$\n\nWe can substitute back into $y+\\frac{1}{x}=29$ to obtain\n$y+\\frac{1+y}{5}=29$\n$5y+1+y=145$\n$y=24.$\n\nWe can then substitute once again to get\n$x=\\frac15$\n$z=\\frac{5}{24}.$\nThus, $z+\\frac1y=\\frac{5}{24}+\\frac{1}{24}=\\frac{1}{4}$, so $m+n=005$.*)\n\nFormal:\ntheorem\n  fixes x y z :: real\n    and p :: rat\n  assumes \"0 < x \\<and> 0 < y \\<and> 0 < z\"\n    and \"x * y * z = 1\"\n    and \"x + 1 / z = 5\"\n    and \"y + 1 / x = 29\"\n    and \"z + 1 / y = p\"\n    and \"0 < p\" \n  shows \"let (m,n) = quotient_of p in m + n = 5\"\nproof -\n  (* We can rewrite $xyz=1$ as $\\frac{1}{z}=xy$. *)\n  have c0: \"z = 1 / (x*y)\"\n    sledgehammer\n  (* Substituting into one of the given equations, we have \n  $x+xy=5$\n  $x(1+y)=5$\n  $\\frac{1}{x}=\\frac{1+y}{5}.$ *)\n  have c1: \"1 / x = (1+y) / 5\" \n  proof -\n    have \"x + x * y = 5\" using assms(3) unfolding c0\n      sledgehammer\n    then have \"x * (1 + y) = 5\"\n      sledgehammer\n    then have t1: \"x = 5 / (1+y)\"\n      sledgehammer\n    then show ?thesis\n      sledgehammer\n  qed\n  (* We can substitute back into $y+\\frac{1}{x}=29$ to obtain\n  $y+\\frac{1+y}{5}=29$\n  $5y+1+y=145$\n  $y=24.$ *)\n  have \"y + (1+y)/5 = 29\" using assms(4) unfolding c1 sledgehammer\n  then have \"5* (y + (1+y)/5) = 5 * 29\" sledgehammer\n  also have \"... = 145\" sledgehammer\n  finally have c2_1: \"5* (y + (1+y)/5) = 145\" sledgehammer\n  have \"5* (y + (1+y)/5) = 5*y + (1+y)\" sledgehammer\n  also have \"... = 6*y + 1\" sledgehammer\n  finally have c2_2: \"5* (y + (1+y)/5) = 6*y + 1\" sledgehammer\n  have \"6*y + 1 = 145\" using c2_1 c2_2 sledgehammer\n  then have c2: \"y = 24\" sledgehammer\n  (* We can then substitute once again to get\n  $x=\\frac15$\n  $z=\\frac{5}{24}.$ *)\n  have \"1/x = 5\" using c1 unfolding c2 sledgehammer\n  then have c3: \"x = 1/5\"\n    sledgehammer\n  then have c4: \"z = 5/24\"\n    sledgehammer\n  (* Thus, $z+\\frac1y=\\frac{5}{24}+\\frac{1}{24}=\\frac{1}{4}$, so $m+n=005$. *)\n  have \"p = z + 1/y\" using assms(5) sledgehammer\n  also have \"... = 5/24 + 1/24\" unfolding c2 c4 sledgehammer\n  also have \"... = 1/4\" sledgehammer\n  finally have c5: \"p = 1/4\"\n    sledgehammer\n  have \"quotient_of p = (1, 4)\" unfolding c5 sledgehammer\n  then show ?thesis sledgehammer\nqed"},
    {"tag": "algebra_2rootsintpoly_am10tap11eqasqpam110", "category": "algebra", "metadata": {}, "prompt": "Informal:\n(*### Problem\n\nShow that for any complex number a, $(a-10)(a+11) = a^2 + a - 110$.\n\n### Solution\n\nWe first expand all terms of the left hand side to get $a^2 - 10a + 11a - 10*11$.\nThis equals $a^2 + a - 10*11 = a^2 + a - 110$.*)\n\nFormal:\ntheorem\n  fixes a :: complex\n  shows \"(a-10) * (a+11) = a^2 + a -110\"\nproof -\n  (* We first expand all terms of the left hand side to get $a^2 - 10a + 11a - 10*11$. *)\n  have \"(a-10) * (a+11) = a^2 - 10*a + 11*a - 10 *11\"\n    sledgehammer\n  (* This equals $a^2 + a - 10*11 = a^2 + a - 110$. *)\n  also have \"\\<dots> = a^2 + a - 10 * 11\"\n    sledgehammer\n  also have \"\\<dots> = a^2 + a - 110\"\n    sledgehammer\n  finally show ?thesis\n    sledgehammer\nqed"},
    {"tag": "mathd_numbertheory_335", "category": "number_theory", "metadata": {}, "prompt": "Informal:\n(*### Problem\n\nWhen Rachel divides her favorite number by 7, she gets a remainder of 5. What will the remainder be if she multiplies her favorite number by 5 and then divides by 7? Show that it is 4.\n\n### Solution\n\nLet $n$ be Rachel's favorite number. \nThen $n \\equiv 5 \\pmod{7}$, so $5n \\equiv 5 \\cdot 5 \\equiv 25 \\equiv 4 \\pmod{7}$.\n*)\n\nFormal:\ntheorem\n  fixes n :: nat\n  assumes h0 : \"n mod 7 = 5\"\n  shows \"(5 * n) mod 7 = 4\"\nproof -\n  (* Then $n \\equiv 5 \\pmod{7}$, so $5n \\equiv 5 \\cdot 5 \\equiv 25 \\equiv 4 \\pmod{7}$. *)\n  have c0:\"(5 * n) mod 7 = (5 * 5) mod 7\" using h0\n    sledgehammer\n  then have \"\\<dots> = 4\" sledgehammer\n  then have \"(5 * n) mod 7 = 4\" using c0 sledgehammer\n  then show ?thesis sledgehammer\nqed"}
]

## Draft

This function generates an *informal* proof $y_{I}\sim p(\cdot|x_I,P_{\text{draft}})$, called a *draft*.

Here, $P_{\text{draft}}$ is a prompt containing examples of mapping the informal theorem statement $x_I$ to an informal proof $y_I$:


In [ ]:
prompt = """Draft an informal solution similar to below. 
The informal solution will be used to sketch a formal Isabelle proof.
Here are some examples:
"""
for x in examples:
    prompt += ("Example:\n" + x['prompt'][:x['prompt'].find('Formal:')] + "\n\n")
prompt += """Informal:
(*### Problem

"""

print(prompt)

In [ ]:
import dsp_functions

In [ ]:
p = dsp_functions.LMFunction('gpt-4')
xi = 'Show that if x is even, then x+5 is odd'
yi = p.f(prompt, xi)
print(yi)

## Sketch

Generate a *formal sketch* $z_{F}\sim p(\cdot|y_{I}, x_I, x_F, P_{\text{sketch}})$

Here, $P_{\text{sketch}}$ is a prompt containing the examples from the drafting step with an additional formal sketch.

In [ ]:
prompt = """Translate the informal solution into a sketch of the
formal Isabelle proof. Add `sledgehammer` in the sketch whenever
possible. `sledgehammer` will be used to call the automated Sledgehammer prover. 
Here are some examples:
"""
for x in examples:
    prompt += (x['prompt'] + "\n\n")
prompt += """Informal:
(*### Problem

"""

xf = """theorem
fixes x :: int
assumes h0: "even x"
shows "odd (x+5)" """

zi = p.f(prompt, xi + '\n\n' + yi + '\n\n' + xf)
print(zi)

## Proof

Finally, we call [Sledgehammer](https://isabelle.in.tum.de/website-Isabelle2009-1/sledgehammer.html) to prove the remaining intermediate conjectures.

You can see the completed proof printed in the cell output:


In [ ]:
theorem_with_proof = """theorem
fixes x :: int
assumes h0: "even x"
shows "odd (x+5)"
proof -
  (* If x is even, then it can be represented as 2n where n is some integer. *)
  obtain n where c1: "x = 2*n"
    using evenE assms
    sledgehammer
  (* So x + 5 equals 2n + 5. *)
  then have "x + 5 = 2*n + 5" 
    sledgehammer
  (* We can rewrite 2n + 5 as 2(n + 2) + 1, which is in the form of 2k + 1 where k is an integer (in this case, n + 2). Thus, x + 5 is odd. *)
  also have "\<dots> = 2*(n+2) + 1"
    sledgehammer
  then have exI: "\<exists>k. x + 5 = 2*k+1" 
    sledgehammer
  then have "odd (x+5)" 
    sledgehammer
  then show ?thesis 
    sledgehammer
qed"""

result = checker.check(theorem_with_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

We now have a verified formal proof of the the claim "If x is even, then x+5 is odd", and the proof is annotated with informal proof steps as comments (the text inside of `(*....*)`. Pretty cool!

## Proof search

In the simple example above, the formal proof was successful on the first try.

In more complex settings we need to try multiple times. 

Namely, we can sample multiple drafts, sample multiple formal sketches for each draft, then see if any of them can be successfully proved with Sledgehammer:

<img src="./images/dsp_search.png" width="800px">

This proof search algorithm is different from the best-first search in next-step suggestion. Namely, it:
1. narrows the search space to proofs that are "similar to" the informal proof\*
2. does not interact with the proof assistant after each step

\*Though ultimately it is up to the neural language model to decide how to use the informal proof (if at all).

### Scaling up proof search

The Draft, Sketch, Prove paper shows the effect of scaling up proof search; that is, sampling multiple drafts and/or multiple formal sketches and attempting to verify them.

Namely, proof search with a single sampled sequence yielded less than 100 successful proofs, but scaling to 100 sampled drafts and/or sketches yielded almost 200 successful proofs:

<img src="./images/dsp_plot.png" width="500px">

Naturally, a better model would require a smaller search budget when measured in terms of number of calls to the model. For instance, imagine a very good model that proved the same ~200 theorems on its first try. This would require fewer calls to the model than above.

Second, the search algorithm used was fairly naive (temperature sampling that only interacts with the proof assistant once). A better search algorithm could yield more successful proofs, and/or a smaller search budget to reach a given number of successful proofs.

#### Examples

Here is an interesting example from the Draft, Sketch, Prove paper. It shows how the neural model can generate an informal proof that differs from the human-written informal proof. The resulting sketches, and formal proofs produced by the system are much different:

<img src="images/dsp_example.png" width="600px">

Feel free to play around with the gpt-4 based implementation above to see what it can do.

----------------------

# Other cascades

[Baldur [First et al 2023]](https://arxiv.org/pdf/2303.04910.pdf) develop a **refinement**, or *proof repair*, module to correct proofs using  error messages $e$:

- $y_F^1\sim p(\cdot|x_F)$
- $y_F^{2}\sim p(\cdot|x_F,y_F^1,e)$


[pySagredo [Azerbayev 2023]](https://github.com/zhangir-azerbayev/pySagredo) is an experimental tactic that uses refinement and GPT-4 to prove theorems in Lean4.


More generally, integrating modern language models with proof assistants remains an active area of research.